# Convolutional network

1. Stride: comment se deplace le kernel sur l'input (image, time series,...). Aussi definit comme la distance qui sépare 2 kernels consecutifs
2. Padding: comment se comporte le filtre en bordure de l'input: 
    * no padding, le filtre s'arrete des qu'il rencontre le bord de l'image
    * zero padding: des zeros sont concatené à l'image en bordure
3. quand le stride est unitaire: astuce pour trouver la dimension de sortie: (input shape-kernel_size+1, nbre de kernel)
     * no zero padding, units stride: $o= (i-k)+1$
     * zero padding p, units stride: $o=(i+2p-k)+1$
     * half/same padding: on veut output size =input size $o=i$: k impaire $k=2n+1$, $p=floor(k/2)=n$ alors $o=i+2p-(k-1)=i$
     * full padding (effet de up-sampling)$p=k-1$
4. Quand le stide s n'est plus unitaire:
    * No zero padding, non unit stride: $o=floor((i-k)/s)+1$
    * Zero padding, non units stride
4. Accelerer le down-sampling: utiliser des Pooling layer (maxpooling,...)
5. Le padding est souvent utiliser pour contrer l'effet du downsampling

## Data structure and remark

1. Image, multi dimensional time series: same structure: 
  * one dimesension for what matter (time axis, image dimension,...), 
  * one other called 'channel axis': red-blue-green for image, left and right channel)
2. convolution = cross-correlation (from signal processing percptective)
3. Operation pour faire la convolution: multiplication de l'output par une matrice de convolution qui est sparse (avec des zeros). On dit transpose convolution car on utlise la transposé de la matrice de convolution


In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers.merge import _Merge
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D, MaxPooling1D, GlobalAveragePooling1D, MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D, Conv1D,Conv2DTranspose
from keras.models import Sequential, Model
from keras.optimizers import RMSprop
from functools import partial

import keras.backend as K

import matplotlib.pyplot as plt

import sys

import numpy as np

Using TensorFlow backend.


# 1D Conv network

1. 1st argument: nbre of kernel/filter
2. 2nd argument: size of the kernel 
3. remark: stride=1 by default, no padding : le filtre se deplace de 1 en 1 partant du haut de la matrice vers le bas

Dans cette example, avec 4 convolution 1D, on passe d'un input 2D de taille (80,3) à un output (44,160).

Donc on fait du down-sampling

https://blog.goodaudience.com/introduction-to-1d-convolutional-neural-networks-in-keras-for-time-sequences-3a7ff801a2cf

In [3]:
TIME_PERIODS=80
num_sensors=3
input_shape=(80,3)


model_m = Sequential()
#model_m.add(Reshape((TIME_PERIODS, num_sensors), input_shape=(input_shape,)))
model_m.add(Conv1D(100, 10, activation='relu', input_shape=(TIME_PERIODS, num_sensors)))
model_m.add(Conv1D(100, 12, activation='relu'))
model_m.add(MaxPooling1D(3))
model_m.add(Conv1D(160, 10, activation='relu'))
model_m.add(Conv1D(160, 10, activation='relu'))
print(model_m.summary())

W0927 11:10:39.198929  7508 deprecation_wrapper.py:119] From C:\Users\rochej\AppData\Local\Continuum\anaconda3\envs\ml_env\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0927 11:10:39.224859  7508 deprecation_wrapper.py:119] From C:\Users\rochej\AppData\Local\Continuum\anaconda3\envs\ml_env\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0927 11:10:39.233809  7508 deprecation_wrapper.py:119] From C:\Users\rochej\AppData\Local\Continuum\anaconda3\envs\ml_env\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0927 11:10:39.275760  7508 deprecation_wrapper.py:119] From C:\Users\rochej\AppData\Local\Continuum\anaconda3\envs\ml_env\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 71, 100)           3100      
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 60, 100)           120100    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 20, 100)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 11, 160)           160160    
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 2, 160)            256160    
Total params: 539,520
Trainable params: 539,520
Non-trainable params: 0
_________________________________________________________________
None


### Conditioning

In [53]:
b=40
k=6
input_shape=(b,k)
kernel_length=5


model_m = Sequential()
model_m.add(Conv1D(k, kernel_length, strides=2, activation='relu', input_shape=(b, k),padding='same'))
model_m.add(Conv1D(k, kernel_length, strides=2, activation='relu',padding='same'))
model_m.add(Conv1D(k, kernel_length, strides=2, activation='relu',padding='same'))
model_m.add(Conv1D(k, kernel_length, strides=2, activation='relu',padding='same'))
model_m.add(Flatten())
model_m.add(Dense(k, activation='relu'))
print(model_m.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_99 (Conv1D)           (None, 20, 6)             186       
_________________________________________________________________
conv1d_100 (Conv1D)          (None, 10, 6)             186       
_________________________________________________________________
conv1d_101 (Conv1D)          (None, 5, 6)              186       
_________________________________________________________________
conv1d_102 (Conv1D)          (None, 3, 6)              186       
_________________________________________________________________
flatten_9 (Flatten)          (None, 18)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 6)                 114       
Total params: 858
Trainable params: 858
Non-trainable params: 0
_________________________________________________________________
None


### Simulator

1. Input shape=(4,k) result of the concatenation of:
   * output du dense: (1,k)
   * noise vector : (2,k)
   * vector A: (1,k)

In [55]:
def Conv1DTranspose(input_tensor, filters, kernel_size, strides=2, padding='same'):
    x = Lambda(lambda x: K.expand_dims(x, axis=2))(input_tensor)
    x = Conv2DTranspose(filters=filters, kernel_size=(kernel_size, 1), strides=(strides, 1), padding=padding)(x)
    x = Lambda(lambda x: K.squeeze(x, axis=2))(x)
    return x

In [65]:
f=20
print(k*f)

model_s = Sequential()
model_s.add(Dense(f*k, activation='relu',input_shape=(4*k,)))
model_s.add(Reshape((int(f/4), int(4*k))))
model_s.add(Reshape(( -1, int(f/4),int(4*k))))
model_s.add(Conv2DTranspose(int(2*k),kernel_size=(1,5),strides=(1,2),padding='same'))
model_s.add(Conv2DTranspose(int(k),kernel_size=(1,5),strides=(1,2),padding='same'))
model_s.add(Reshape((int(f), int(k))))
model_s.summary()





120
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 120)               3000      
_________________________________________________________________
reshape_22 (Reshape)         (None, 5, 24)             0         
_________________________________________________________________
reshape_23 (Reshape)         (None, 1, 5, 24)          0         
_________________________________________________________________
conv2d_transpose_13 (Conv2DT (None, 1, 10, 12)         1452      
_________________________________________________________________
conv2d_transpose_14 (Conv2DT (None, 1, 20, 6)          366       
_________________________________________________________________
reshape_24 (Reshape)         (None, 20, 6)             0         
Total params: 4,818
Trainable params: 4,818
Non-trainable params: 0
_________________________________________________________________


### Discriminator

In [41]:
model_d = Sequential()
model_d.add(Conv1D(2*k, kernel_length, strides=2, activation='relu', input_shape=(60, k),padding='same'))
model_d.add(Conv1D(2*2*k, kernel_length, strides=2, activation='relu', input_shape=(60, k),padding='same'))
model_d.add(Conv1D(2*2*2*k, kernel_length, strides=2, activation='relu', input_shape=(60, k),padding='same'))
model_d.add(Conv1D(2*2*2*2*k, kernel_length, strides=2, activation='relu', input_shape=(60, k),padding='same'))
model_d.add(Conv1D(2*2*2*2*2*k, kernel_length, strides=2, activation='relu', input_shape=(60, k),padding='same'))
model_d.add(Flatten())
print(model_d.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_94 (Conv1D)           (None, 30, 12)            372       
_________________________________________________________________
conv1d_95 (Conv1D)           (None, 15, 24)            1464      
_________________________________________________________________
conv1d_96 (Conv1D)           (None, 8, 48)             5808      
_________________________________________________________________
conv1d_97 (Conv1D)           (None, 4, 96)             23136     
_________________________________________________________________
conv1d_98 (Conv1D)           (None, 2, 192)            92352     
_________________________________________________________________
flatten_8 (Flatten)          (None, 384)               0         
Total params: 123,132
Trainable params: 123,132
Non-trainable params: 0
_________________________________________________________________
None

In [37]:
2**5*6

192

ici, on fait de la convolution sur le temps

# 2D Conv network

1. remarque: input shape: toujours en 3D meme pour une matrice 2D (example, si input est une matrice (4,4), il faut rentrer (4,4,1)
2. Par defaut, stride de (1,1): se deplace de 1 en 1 verticalement 
3. A l'inverse, stride de (2,2): meme effet que des pooling layer cad down-sampling
On fait une reduction de dimension


Operation interne à l'aide d'une matrice de convolution
1. On construit la matrice de convolution à parir du kernel

https://medium.com/activating-robotic-minds/up-sampling-with-transposed-convolution-9ae4f2df52d0

https://www.pyimagesearch.com/2018/12/31/keras-conv2d-and-convolutional-layers/

In [62]:
model = Sequential()
model.add(Conv2D(5, kernel_size=(1,1), strides=(2, 2), activation='relu',input_shape=(4,4,1)))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_54 (Conv2D)           (None, 2, 2, 5)           10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________
None


In [63]:
dim_image=(20,20)
kernel_size=(3,3)

data=np.ones((20,20))

model = Sequential()
model.add(Conv2D(100, kernel_size, strides=(1, 1), activation='relu',input_shape=(dim_image[0],dim_image[1],1)))
model.add(Conv2D(200, kernel_size, strides=(2, 2), activation='relu'))
model.add(Conv2D(200, kernel_size, strides=(1, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_55 (Conv2D)           (None, 18, 18, 100)       1000      
_________________________________________________________________
conv2d_56 (Conv2D)           (None, 8, 8, 200)         180200    
_________________________________________________________________
conv2d_57 (Conv2D)           (None, 6, 6, 200)         360200    
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 3, 3, 200)         0         
Total params: 541,400
Trainable params: 541,400
Non-trainable params: 0
_________________________________________________________________
None


# Transpose Conv
Goal: upsampling as for DCGAN: take randomly sampled values to produce a full-size image
1. stride refers to the manner in which outputs in the feature map are laid down
2. on up-sample soit avec le stride, soit avec le kernel size
3. La transpose convolution peut etre equivalent à une convolution en jouent sur le zero-padding (possible méthode mais moins efficace)

Code tutorial:
https://machinelearningmastery.com/upsampling-and-transpose-convolution-layers-for-generative-adversarial-networks/

Introduction without code:
https://medium.com/activating-robotic-minds/up-sampling-with-transposed-convolution-9ae4f2df52d0

https://medium.com/apache-mxnet/transposed-convolutions-explained-with-ms-excel-52d13030c7e8

In [70]:
model = Sequential()
model.add(Conv2DTranspose(5, kernel_size=(1,1), strides=(2,2), input_shape=(2, 2, 1)))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_transpose_8 (Conv2DTr (None, 4, 4, 5)           10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


In [71]:
model = Sequential()
model.add(Conv2DTranspose(5, kernel_size=(3,3), strides=(1,1), input_shape=(2, 2, 1)))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_transpose_9 (Conv2DTr (None, 4, 4, 5)           50        
Total params: 50
Trainable params: 50
Non-trainable params: 0
_________________________________________________________________


Avec le padding='same', on force l'output de l'operation a avoir la même dimension que l'input


In [72]:
model = Sequential()
model.add(Conv2DTranspose(1, (3,3), strides=(2,2), padding='same', input_shape=(2, 2, 1)))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_transpose_10 (Conv2DT (None, 4, 4, 1)           10        
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


Example d'un generateur construit avec Transpose convolution

1. Goal: built an image of dimension: 10x10 
2. Noise input: vector of 100 elements
3. Avec un Dense layer: noise est transformé en 128 'images' de taille 5x5 qu'on va chercher à up-sampler en image de taille 10x10

In [82]:
model = Sequential()
model.add(Dense(128 * 5 * 5, input_dim=100))
model.add(Reshape((5, 5, 128)))
model.add(Conv2DTranspose(1, (3,3), strides=(2,2),padding='same'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 3200)              323200    
_________________________________________________________________
reshape_6 (Reshape)          (None, 5, 5, 128)         0         
_________________________________________________________________
conv2d_transpose_18 (Conv2DT (None, 10, 10, 1)         1153      
Total params: 324,353
Trainable params: 324,353
Non-trainable params: 0
_________________________________________________________________


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 3200)              323200    
_________________________________________________________________
reshape_5 (Reshape)          (None, 5, 5, 128)         0         
_________________________________________________________________
conv2d_transpose_16 (Conv2DT (None, 11, 11, 1)         1153      
_________________________________________________________________
conv2d_transpose_17 (Conv2DT (None, 22, 22, 1)         10        
Total params: 324,363
Trainable params: 324,363
Non-trainable params: 0
_________________________________________________________________


# Keras

1. train_on_batch(x,y): 
    * Prend 2 argument x: training data et y: target value (labels)
    * Runs a single gradient update on a single batch of data
    * Return scalar training loss/ list of training loss (if the model has sevral outputs)

In [35]:
noise=np.random.normal(0,1,(3,16)) #3 input de taille 16
noise.shape

(3, 16)

In [36]:
#model_simple
model = Sequential()
model.add(Dense(10,input_shape=(16,)))
model.add(Dense(2))

model.predict(noise)


array([[-0.10309774,  0.5624553 ],
       [ 0.48000252, -1.9803181 ],
       [-1.0735989 ,  0.12747216]], dtype=float32)

In [47]:
weights_1, biases_1 = model.layers[0].get_weights()
weights_2, biases_2 = model.layers[1].get_weights()

In [48]:
print(weights_1.shape)
print(weights_2.shape)

(16, 10)
(10, 2)


In [51]:
#accès a la configuration de tout le reseau
model.get_config()
#acces au nombre de parametre optimisable dans la premiere courche= dimension de la  matrice + 10 biaises
print(model.layers[0].count_params())

170


In [56]:
#model_convolutional
#k:nbre of filter
k=10
kernel_length=5
b=40
noise=np.random.normal(0,1,(3,b,k))# 3 input de taille b,k
model_c = Sequential()
model_c.add(Conv1D(k, kernel_length, strides=2, activation='relu', input_shape=(b, k),padding='same'))
model_c.add(Flatten())
model_c.add(Dense(2, activation='relu'))


model_c.predict(noise).shape



(3, 2)

In [64]:
weights_1, biases_1 = model_c.layers[0].get_weights()
print(weights_1.shape)
print(biases_1.shape) #egale au nombre de filtre

(5, 10, 10)
(10,)


In [66]:
w=model_c.layers[0].get_weights()

In [77]:
#eval tensor operation with numpy array
real_noise=np.ones((3,10))*4
tensor_noise=K.variable(real_noise)
square_v=K.square(tensor_noise)
K.eval(square_v)

array([[16., 16., 16., 16., 16., 16., 16., 16., 16., 16.],
       [16., 16., 16., 16., 16., 16., 16., 16., 16., 16.],
       [16., 16., 16., 16., 16., 16., 16., 16., 16., 16.]], dtype=float32)

array([[4., 4., 4., 4., 4., 4., 4., 4., 4., 4.],
       [4., 4., 4., 4., 4., 4., 4., 4., 4., 4.],
       [4., 4., 4., 4., 4., 4., 4., 4., 4., 4.]])